In [1]:
import pandas as pd

In [2]:
train_labels = pd.read_csv("Data/train.csv")

In [3]:
train_keys = dict([a[1] for a in train_labels.iterrows()])

In [4]:
test_files = set(pd.read_csv('Data/sampleSubmission.csv').file.values)

In [5]:
import json
PATH_TO_DATA = "/Users/Nerrtica/Documents/Study/3rdYear/SWMaestro/2ndProject/Kaggle/TrulyNative?/Data/"

In [6]:
import re

# put in the path to the kaggle data
PATH_TO_JSON = "Data/parsed_data"
PATH_TO_TRAIN_LABELS = "Data/train.csv"
PATH_TO_TEST_LABELS = "Data/sampleSubmission.csv"

# a simple method to create some basic features on an SFrame
def create_count_features(sf):
    sf['num_images'] = sf['images'].apply(lambda x: len(x))
    sf['num_links'] = sf['links'].apply(lambda x: len(x))
    sf['num_clean_chars'] = sf['text_clean'].apply(lambda x: len(x))
    return sf

# a simple method to clean the text within an html response
def clean_text(sf):
    sf['text_clean'] = sf['text'].apply(lambda x:
        #re.sub(r'[\n\t,.:;()\-\/]+', ' ', ' '.join(x)))
        re.sub('[^a-zA-Z]', ' ', ' '.join(x)))
    sf['text_clean'] = sf['text_clean'].apply(lambda x: re.sub(r'\s{2,}', ' ', x))
    sf['text_clean'] = sf['text_clean'].apply(lambda x: x.strip())
    return sf

# a wrapper method around the 2 methods above
def process_dataframe(sf):
    sf = clean_text(sf)
    sf = create_count_features(sf)
    return sf 

In [1]:
import graphlab as gl

ImportError: No module named graphlab

In [8]:
# read json blocks from path PATH_TO_JSON
sf = gl.SFrame.read_csv(PATH_TO_JSON, header=False)

[INFO] This trial license of GraphLab Create is assigned to nerrtica@naver.com and will expire on November 01, 2015. Please contact trial@dato.com for licensing options or to request a free non-commercial license for personal or academic use.

[INFO] Start server at: ipc:///tmp/graphlab_server-673 - Server binary: /Users/Nerrtica/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1444010490.log
[INFO] GraphLab Server Version: 1.6.1


PROGRESS: Finished parsing file /Users/Nerrtica/Documents/Study/3rdYear/SWMaestro/2ndProject/Kaggle/TrulyNative?/Data/parsed_data/chunk0.json
PROGRESS: Parsing completed. Parsed 100 lines in 0.654601 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[dict]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Read 1936 lines. Lines per second: 1651.18
PROGRESS: Read 15153 lines. Lines per second: 2419.7
PROGRESS: Read 29470 lines. Lines per second: 2540.09
PROGRESS: Read 43765 lines. Lines per second: 2540.51
PROGRESS: Read 57315 lines. Lines per second: 2576.51
PROGRESS: Finished parsing file /Users/Nerrtica/Documents/Study/3rdYear/SWMaestro/2ndProject/Kaggle/TrulyNative?/Data/parsed_data/chunk0.json
PROGRESS: Read 69376 lines. Lines per second: 2534.89
PROGRESS

In [9]:
sf = sf.unpack('X1',column_name_prefix='')

In [10]:
# read train and test labels from paths PATH_TO_TRAIN_LABELS and PATH_TO_TEST_LABELS
train_labels = gl.SFrame.read_csv(PATH_TO_TRAIN_LABELS)
test_labels = gl.SFrame.read_csv(PATH_TO_TEST_LABELS)

PROGRESS: Finished parsing file /Users/Nerrtica/Documents/Study/3rdYear/SWMaestro/2ndProject/Kaggle/TrulyNative?/Data/train.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.277896 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/Nerrtica/Documents/Study/3rdYear/SWMaestro/2ndProject/Kaggle/TrulyNative?/Data/train.csv
PROGRESS: Parsing completed. Parsed 337024 lines in 0.225021 secs.
PROGRESS: Finished parsing file /Users/Nerrtica/Documents/Study/3rdYear/SWMaestro/2ndProject/Kaggle/TrulyNative?/Data/sampleSubmission.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.057248 secs.
------------------------------------------------------
Inferred types from first line

In [11]:
# create a new columns "id" from parsing urlId and drop file columns
train_labels['id'] = train_labels['file'].apply(lambda x: str(x.split('_')[0] ))
train_labels = train_labels.remove_column('file')
test_labels['id'] = test_labels['file'].apply(lambda x: str(x.split('_')[0] ))
test_labels = test_labels.remove_column('file')

In [12]:
# join labels with html data from training and testing SFrames
train = train_labels.join(sf, on='id', how='left')
test = test_labels.join(sf, on='id', how='left')

In [13]:
# call wrapper method process_dataframe on train/test
train = process_dataframe(train)
test = process_dataframe(test)

In [14]:
import re
import nltk
from nltk.corpus import stopwords

In [15]:
bow_trn = gl.text_analytics.count_words(train['text_clean'])
bow_trn = bow_trn.dict_trim_by_keys(gl.text_analytics.stopwords())

In [16]:
train['bow'] = bow_trn

In [17]:
bow_tst = gl.text_analytics.count_words(test['text_clean'])
bow_tst = bow_tst.dict_trim_by_keys(gl.text_analytics.stopwords())

test['bow'] = bow_tst

In [18]:
from graphlab.toolkits.feature_engineering import TFIDF

In [19]:
# create a TFIDF Transformer that is fit on your training data and transform both training and testing data
encoder = gl.feature_engineering.create(train, TFIDF('bow', output_column_name='tfidf'))
train = encoder.transform(train)
test = encoder.transform(test)

In [20]:
# from sklearn.ensemble import RandomForestClassifier

In [21]:
# rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)

In [28]:
train = train.dropna()

In [ ]:
model = gl.random_forest_classifier.create(train, num_trees = 100, target = 'sponsored', features = ['tfidf'], class_weights = 'auto', validation_set = None)

In [30]:
test = test.dropna()
ypred = model.predict(test, output_type = 'probability')

In [37]:
# create submission.csv
submission = gl.SFrame()
submission['sponsored'] = ypred 
submission['file'] = test['id'].apply(lambda x: x + '_raw_html.txt')
submission.save('Data/result.csv', format='csv')